# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

import nltk
import pickle
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matthew.Hull\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matthew.Hull\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Matthew.Hull\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterMessagesDatabase.db')
df = pd.read_sql_table("DisasterMessagesTable", con=engine)


In [3]:
df.head(1)

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df["genre"].value_counts()

news      13053
direct    10766
social     2395
Name: genre, dtype: int64

In [5]:
X = df["message"]
y = df.drop(columns=["message", "original", "genre"])

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
        
    tokens = word_tokenize(text,language='english',)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    
    removed_stop_words = [tok for tok in clean_tokens if tok not in stopwords.words("english")]
    
    return removed_stop_words


In [7]:
X.iloc[10]

"There's nothing to eat and water, we starving and thirsty."

In [8]:
tokenize(X[10])

['nothing', 'eat', 'water', 'starving', 'thirsty']

In [9]:
y.iloc[10]

related                   1
request                   1
offer                     0
aid_related               1
medical_help              1
medical_products          1
search_and_rescue         0
security                  0
military                  0
water                     1
food                      1
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 1
infrastructure_related    1
transport                 1
buildings                 1
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      1
weather_related           1
floods                    1
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report             1
Name: 10, dtype: int

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024AA9A939D0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7137,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8533,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3623,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
673,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21311,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12338,1,0,0,1,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,1
24691,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18126,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5132,1,1,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,1


In [14]:
y_test.shape

(6554, 35)

In [15]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
y_pred.shape

(6554, 35)

In [17]:
y_test.iloc[0]

related                   1
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report             0
Name: 7137, dtype: i

In [18]:
y_test.iloc[:,0].unique()

array([1, 0], dtype=int64)

In [19]:
for i in range(len(y_train.columns)):
    print(f"For {y_train.columns[i]}:")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print("-"*50)


For related:
              precision    recall  f1-score   support

           0       0.62      0.45      0.52      1485
           1       0.85      0.92      0.88      5069

    accuracy                           0.81      6554
   macro avg       0.74      0.68      0.70      6554
weighted avg       0.80      0.81      0.80      6554

--------------------------------------------------
For request:
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5429
           1       0.79      0.43      0.55      1125

    accuracy                           0.88      6554
   macro avg       0.84      0.70      0.74      6554
weighted avg       0.87      0.88      0.87      6554

--------------------------------------------------
For offer:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99

c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: P

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6470
           1       0.50      0.01      0.02        84

    accuracy                           0.99      6554
   macro avg       0.74      0.51      0.51      6554
weighted avg       0.98      0.99      0.98      6554

--------------------------------------------------
For refugees:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6333
           1       0.70      0.09      0.15       221

    accuracy                           0.97      6554
   macro avg       0.84      0.54      0.57      6554
weighted avg       0.96      0.97      0.96      6554

--------------------------------------------------
For death:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6246
           1       0.76      0.18      0.29       308

    accuracy                           0.96      6554
 

c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: P

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000024AA9A939D0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000024AA9A939D0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True

In [22]:
parameters = {
    'clf__estimator__max_depth': [5, 10],
    'clf__estimator__min_samples_leaf': [1,2]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=1; total time= 4.5min


c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=1; total time= 4.8min


...

c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=1; total time= 3.7min


c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=1; total time= 3.2min


c:\Users\Matthew.Hull\nap_local\nanodegree_datascience_term_2\udacity_ds_term2_disaster_response_pipeline\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

for i in range(len(y_train.columns)):
    print(f"For {y_train.columns[i]}:")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print("-"*50)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Tried to use Linear SVC (as per https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) but was unable to get it working in the time limit

### 9. Export your model as a pickle file

In [ ]:
best_model = cv.best_estimator_

with open("best_model.pickle", "w") as f:
    pickle.dump(best_model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.